In [ ]:
import json
import os
import sys

import requests
from dotenv import load_dotenv

load_dotenv()


def fetch_and_save_data(endpoint: str, output_file: str):
    """
    Fetches data from a specified API endpoint and saves it to a JSON file.

    :param endpoint: The API endpoint to fetch data from (e.g., "api/v0/equity/metadata/instruments").
    :param output_file: The name of the file to save the data (e.g., "instrument_list_data.json").
    """
    try:
        # Build the full URL
        base_url = os.environ.get("TRADING212_HOST", "demo.trading212.com")

        if not os.environ.get("TRADING212_API_KEY"):
            print("Error: Missing TRADING212_API_KEY environment variable.")
            sys.exit(1)

        url = f"https://{base_url}/{endpoint}"
        response = requests.get(
            url,
            headers={"Authorization": os.environ.get("TRADING212_API_KEY")},
            timeout=10,
        )
        response.raise_for_status()

        # Parse JSON response
        data = response.json()

        # Save to file
        with open(output_file, "w") as file:
            json.dump(data, file)
            print(f"Data successfully written to {output_file}.")

    except requests.exceptions.RequestException as e:
        print(f"Error during API request: {e}")
        sys.exit(1)
    except IOError as e:
        print(f"Error writing to file: {e}")
        sys.exit(1)


fetch_and_save_data("api/v0/history/exports", "exports_list_data.json")

In [ ]:
import pandas as pd

data = pd.read_json("exports_list_data.json")
dataframe = pd.DataFrame(data)

print(dataframe.head())

In [ ]:
import pandas as pd


def load_json_file(file_path):
    """Load a JSON file into a Pandas DataFrame."""
    try:
        return pd.read_json(file_path)
    except ValueError as e:
        print(f"Error processing JSON: {e}")
        exit()
    except FileNotFoundError:
        print("The JSON file was not found.")
        exit()


def download_csv(download_link, output_file):
    """Download a CSV file from the specified link."""
    try:
        response = requests.get(download_link)
        response.raise_for_status()  # Check for HTTP errors
        with open(output_file, "wb") as file:
            file.write(response.content)
        print(f"CSV downloaded and saved as: {output_file}")
    except requests.exceptions.RequestException as e:
        print(f"Failed to download the file: {e}")
        exit()
    except IOError:
        print("Error writing the CSV file.")
        exit()


def process_csv(file_path):
    """Load a CSV file into a Pandas DataFrame."""
    try:
        df = pd.read_csv(file_path)
        print(df.head())
    except pd.errors.EmptyDataError:
        print("Error: The CSV file is empty or invalid.")
    except pd.errors.ParserError:
        print("Error: The CSV file format is incorrect.")
    except Exception as e:
        print(f"Unexpected error while processing the CSV file: {e}")


# Main program
json_file_path = "exports_list_data.json"
csv_file_path = "data.csv"

# Load the JSON file
dataframe = load_json_file(json_file_path)

# Extract the download link
download_link = dataframe.iloc[0].get("downloadLink", "")
if not download_link:
    print("The download link is missing.")
    exit()

# Download and process the CSV file
download_csv(download_link, csv_file_path)
process_csv(csv_file_path)